In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torch.optim import SGD
from torchvision import models, datasets, transforms
from torch.nn import functional
import copy
import cv2
import numpy as np

from torch.autograd import Variable
from torchvision import models

"\nproject_id = 'learned-digit-427721-m6'\nfrom google.colab import auth\nauth.authenticate_user()\n!gcloud config set project {project_id}\nbucket_name = 'xbowsniper'\n"

In [ ]:
def preprocess_image(cv2im, resize_im=True):
    """
        Processes image for CNNs

    Args:
        PIL_img (PIL_img): Image to process
        resize_im (bool): Resize to 224 or not
    returns:
        im_as_var (Pytorch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    # Resize image
    if resize_im:
        cv2im = cv2.resize(cv2im, (224, 224))
    im_as_arr = np.float32(cv2im)
    im_as_arr = np.ascontiguousarray(im_as_arr[..., ::-1])
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var


def recreate_image(im_as_var):
    """
        Recreates images from a torch variable, sort of reverse preprocessing

    Args:
        im_as_var (torch variable): Image to recreate

    returns:
        recreated_im (numpy arr): Recreated image in array
    """
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]

    recreated_im = copy.copy(im_as_var.data.numpy()[0])
    for c in range(3):
        recreated_im[c] /= reverse_std[c]
        recreated_im[c] -= reverse_mean[c]
    recreated_im[recreated_im > 1] = 1
    recreated_im[recreated_im < 0] = 0
    recreated_im = np.round(recreated_im * 255)

    recreated_im = np.uint8(recreated_im).transpose(1, 2, 0)
    # Convert RBG to GBR
    recreated_im = recreated_im[..., ::-1]
    return recreated_im


def get_params(example_index):
    """
        Gets used variables for almost all visualizations, like the image, model etc.

    Args:
        example_index (int): Image id to use from examples

    returns:
        original_image (numpy arr): Original image read from the file
        prep_img (numpy_arr): Processed image
        target_class (int): Target class for the image
        file_name_to_export (string): File name to export the visualizations
        pretrained_model(Pytorch model): Model to use for the operations
    """
    # Pick one of the examples
    example_list = [['../input_images/apple.JPEG', 948],
                    ['../input_images/eel.JPEG', 390],
                    ['../input_images/bird.JPEG', 13]]
    selected_example = example_index
    img_path = example_list[selected_example][0]
    target_class = example_list[selected_example][1]
    file_name_to_export = img_path[img_path.rfind('/')+1:img_path.rfind('.')]
    # Read image
    original_image = cv2.imread(img_path, 1)
    # Process image
    prep_img = preprocess_image(original_image)
    # Define model
    pretrained_model = models.alexnet(pretrained=True)
    return (original_image,
            prep_img,
            target_class,
            file_name_to_export,
            pretrained_model)


In [ ]:
class FoolingSampleGeneration:
    """
    Produces an image that maximizes a certain class with gradient ascent, breaks as soon as
    the target prediction confidence is captured
    """
    def __init__(self, model, dataset, target_class, img_idx, minimum_confidence, device):
        self.model = model
        self.model.eval()
        self.dataset = dataset
        self.target_class = target_class
        self.img_idx = img_idx
        self.minimum_confidence = minimum_confidence
        self.device = device
        # Generate a random image
        self.created_image = np.uint8(np.random.uniform(0, 255, (224, 224, 3)))
        # Create the folder to export images if not exists
        if not os.path.exists('../generated'):
            os.makedirs('../generated')
        if not os.path.exists(f'../generated/{self.dataset}/fooling'):
            os.makedirs(f'../generated/{self.dataset}/fooling')

    def generate(self):
        for i in range(1, 200):
            # Process image and return variable
            self.processed_image = preprocess_image(self.created_image).to(self.device).detach().clone()
            self.processed_image.requires_grad = True
            # Define optimizer for the image
            optimizer = SGD([self.processed_image], lr=6)
            # Forward
            output = self.model(self.processed_image)
            # Get confidence from softmax
            target_confidence = functional.softmax(output, dim=1)[0][self.target_class].item()
            if target_confidence > self.minimum_confidence:
                # Reading the raw image and pushing it through model to see the prediction
                # this is needed because the format of preprocessed image is float and when
                # it is written back to file it is converted to uint8, so there is a chance that
                # there are some losses while writing
                confirmation_image_path = os.path.join(f'../generated/{self.dataset}/fooling', f'{self.dataset}_{self.img_idx}_ga_fooling_class_{self.target_class}.jpg')
                confirmation_image = cv2.imread(confirmation_image_path, 1)
                # Preprocess image
                confirmation_processed_image = preprocess_image(confirmation_image).to(self.device)
                # Get prediction
                confirmation_output = self.model(confirmation_processed_image)
                # Get confidence
                softmax_confirmation = functional.softmax(confirmation_output, dim=1)[0][self.target_class].item()
                if softmax_confirmation > self.minimum_confidence:
                    print('Generated fooling image with', "{0:.2f}".format(softmax_confirmation),
                          'confidence at', str(i) + 'th iteration.')
                    break
            # Target specific class
            class_loss = -output[0, self.target_class]
            print('Iteration:', str(i), 'Target Confidence', "{0:.4f}".format(target_confidence))
            # Zero grads
            self.model.zero_grad()
            # Backward
            class_loss.backward()
            # Update image
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(self.processed_image.cpu())  # Move to CPU for saving
            # Save image
            cv2.imwrite(os.path.join(f'../generated/{self.dataset}/fooling', f'{self.dataset}_{self.img_idx}_ga_fooling_class_{self.target_class}.jpg'),
                        self.created_image)
        return self.processed_image

def generate_fooling_images(model, dataset, target_class, num_images, minimum_confidence, device):
    model.to(device)
    for img_idx in range(num_images):
        print(f"Generating fooling image {img_idx + 1} for target class {target_class}")
        fooling_gen = FoolingSampleGeneration(model, dataset, target_class, img_idx, minimum_confidence, device)
        fooling_gen.generate()

if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Generate fooling dataset
    pretrained_model = models.resnet50(pretrained=True)

    target_list = [117,70,88,133,5,97,42,60,14,3,130,57,26,0,89,127,36,67,110,65,123,55,22,21,1,71,99,16,19,108,18,35,124,90,74,129,125,2,64,92,138,48,54,39,56,96,84,73,77,52,20,118,111,59,106,75,143,80,140,11,113,4,28,50,38,104,24,107,100,81,94,41,68,8,66,146,29,32,137,33,141,134,78,150,76,61,112,83,144,91,135,116,72,34,6,119,46,115,93,7]
    # Generate fooling images for various classes
    for i in range(7):
        for target in target_list:
            generate_fooling_images(pretrained_model, f'train{i}', target, 13, 0.9, device)

    for target in target_list:
        generate_fooling_images(pretrained_model, 'validate', target, 30, 0.9, device)

    for target in target_list:
        generate_fooling_images(pretrained_model, 'test', target, 30, 0.9, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Streaming output truncated to the last 5000 lines.
Iteration: 23 Target Confidence 0.9835
Iteration: 24 Target Confidence 0.9891
Iteration: 25 Target Confidence 0.9682
Iteration: 26 Target Confidence 0.9845
Generated fooling image with 0.91 confidence at 27th iteration.
Generating fooling image 17 for target class 119
Iteration: 1 Target Confidence 0.0000
Iteration: 2 Target Confidence 0.0000
Iteration: 3 Target Confidence 0.0000
Iteration: 4 Target Confidence 0.0001
Iteration: 5 Target Confidence 0.0001
Iteration: 6 Target Confidence 0.0001
Iteration: 7 Target Confidence 0.0002
Iteration: 8 Target Confidence 0.0004
Iteration: 9 Target Confidence 0.0009
Iteration: 10 Target Confidence 0.0034
Iteration: 11 Target Confidence 0.0298
Iteration: 12 Target Confidence 0.3648
Iteration: 13 Target Confidence 0.2403
Iteration: 14 Target Confidence 0.8774
Iteration: 15 Target Confidence 0.8808
Iteration: 16 Target Confidence 0.9776
Iteration: 17 Target Confidence 0.9824
Iteration: 18 Target Confi

In [ ]:
!zip -r /content/generatedfooling.zip /generated

Streaming output truncated to the last 5000 lines.
  adding: generated/train5/fooling/train5_5_ga_fooling_class_38.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_12_ga_fooling_class_144.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_0_ga_fooling_class_72.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_11_ga_fooling_class_96.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_0_ga_fooling_class_0.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_1_ga_fooling_class_29.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_10_ga_fooling_class_134.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_1_ga_fooling_class_48.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_10_ga_fooling_class_141.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_10_ga_fooling_class_18.jpg (deflated 1%)
  adding: generated/train5/fooling/train5_10_ga_fooling_class_99.jpg (deflated 1%)
  adding: generated/train5/fooling/trai